<a href="https://colab.research.google.com/github/adi0215/HPE_CTY/blob/main/Callback_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
import mxnet as mx
from mxnet import gluon, autograd, nd
from mxnet.gluon import nn

In [35]:
# Define a simple neural network
net = nn.HybridSequential()
with net.name_scope():
    net.add(nn.Dense(128, activation='relu'))
    net.add(nn.Dense(64, activation='relu'))
    net.add(nn.Dense(10))

ctx=mx.cpu()
net.initialize(ctx=ctx)
net.hybridize()

In [36]:
# Define an optimizer
optimizer = gluon.Trainer(net.collect_params(),'adam')

# Define a training dataset
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=lambda data, label: (data.astype('float32')/255, label)), batch_size=32, shuffle=True)

# Define a loss function
loss_fn = gluon.loss.SoftmaxCrossEntropyLoss()

In [37]:
# Load the MNIST dataset
batch_size = 64
train_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=True, transform=lambda data, label: (data.astype('float32')/255, label)), batch_size, shuffle=True)#splitting data for training
val_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=False, transform=lambda data, label: (data.astype('float32')/255, label)), batch_size, shuffle=False)#splitting data for validation

epoch_num=3

In [46]:
from mxnet.gluon.contrib.estimator import estimator
from mxnet.gluon.contrib.estimator.event_handler import TrainBegin, TrainEnd, EpochEnd

# Define the estimator
estimator = estimator.Estimator(net=net,
                          loss=loss_fn,
                          trainer=optimizer,
                          context=ctx,
                         )

In [47]:
# custom event handler with TrainBegin, TrainEnd, EpochEnd callbacks
class LossRecordHandler(TrainBegin, TrainEnd,EpochEnd):
    def __init__(self):
        super(LossRecordHandler, self).__init__()
        self.loss_history = {}

    def train_begin(self, estimator, *args, **kwargs):
        print("Training starting ")
        
    def train_end(self, estimator, *args, **kwargs):
        # Print all the losses at the end of training
        print("Training ended")
        
    def epoch_end(self, estimator, *args, **kwargs):
        for metric in estimator.train_metrics:
            # look for train Loss in training metrics
            # we wrapped loss value as a metric to record it
            if isinstance(metric, mx.metric.Loss):
                loss_name, loss_val = metric.get()
                # append loss value for this epoch
                self.loss_history.setdefault(loss_name, []).append(loss_val)

In [48]:
# Instantiate Loss record handler 
loss_record_handler = LossRecordHandler()

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
# Train the model with the callback
estimator.fit(train_data, epochs=epoch_num,event_handlers=[loss_record_handler])   

Training starting 
Training begin: using optimizer Adam with current learning rate 0.0010 
Train for 3 epochs.
[Epoch 0] Begin, current learning rate: 0.0010
[Epoch 0] Finished in 13.902s, training accuracy: 0.9933, training softmaxcrossentropyloss: 0.0226
[Epoch 1] Begin, current learning rate: 0.0010
[Epoch 1] Finished in 14.052s, training accuracy: 0.9937, training softmaxcrossentropyloss: 0.0188
[Epoch 2] Begin, current learning rate: 0.0010
[Epoch 2] Finished in 14.244s, training accuracy: 0.9949, training softmaxcrossentropyloss: 0.0160
Training ended
Train finished using total 42s with 3 epochs. training accuracy: 0.9949, training softmaxcrossentropyloss: 0.0160
